<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/static/images/logo-v4.svg" width=190/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/project_configuration/queue_management.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/project_configuration/queue_management.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Queue Management

* The queue is used to task labelers with specific assets
* We can do any of the following:
    * Set quality settings
    * Set the order of items in the queue
    * Set the percent of assets to review

In [2]:
%pip install -q "labelbox[data]"
%pip install -q numpy


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import labelbox as lb
from labelbox.schema.quality_mode import QualityMode
from uuid import uuid4
import json

# API Key and Client
See the developer guide for [creating an API key](https://docs.labelbox.com/reference/create-api-key).

In [4]:
# Add your API key
API_KEY = ""
client = lb.Client(api_key=API_KEY)

### Set up demo project

#### Create project

In [5]:
# Create Labelbox  project

project = client.create_project(
    name="batch-test-project",
    description="a description",
    quality_mode=QualityMode.
    Benchmark,  # For Consensus projects use quality_mode = QualityMode.Consensus
    media_type=lb.MediaType.Image,
)

dataset = client.create_dataset(name="queue_dataset")

#### Create ontology and attach to project

In [6]:
classification_features = [
    lb.Classification(
        class_type=lb.Classification.Type.CHECKLIST,
        name="Quality Issues",
        options=[
            lb.Option(value="blurry", label="Blurry"),
            lb.Option(value="distorted", label="Distorted"),
        ],
    )
]

ontology_builder = lb.OntologyBuilder(
    tools=[], classifications=classification_features
)

ontology = client.create_ontology(
    "Ontology from new features",
    ontology_builder.asdict(),
    media_type=lb.MediaType.Image,
)

project.setup_editor(ontology)

# Add data to your dataset

In [7]:
## Example image
uploads = []
global_keys = []
# Generate data rows
for i in range(1, 5):
    global_key = str(uuid4())
    row = {
        "row_data":
            f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key":
            global_key,
    }
    global_keys.append(global_key)
    uploads.append(row)

data_rows = dataset.create_data_rows(uploads)
data_rows.wait_till_done()
print("Errors", data_rows.errors)
print("Dataset status: ", data_rows.status)

Errors None
Dataset status:  COMPLETE


# Attach data to your project and set data row priority

In [8]:
######## Create batches

# Create the batch

batch = project.create_batch(
    "batch-demo",  # Each batch in a project must have a unique name
    global_keys=global_keys[
        0:2],  # A list of data rows, data row ids or global keys
    priority=
    5,  # priority between 1(Highest) - 5(lowest) 5 is the max priority that can be set
)

batch2 = project.create_batch(
    "batch-demo-2",  # Each batch in a project must have a unique name
    # Provide a slice of the data since you can't import assets with global keys that already exist in the project.
    global_keys=global_keys[
        2:4],  # A list of data rows, data row ids or global keys
    priority=
    1,  # priority between 1(Highest) - 5(lowest) 5 is the max priority that can be set
)

print("Batch: ", batch)
print("Batch2: ", batch2)

Batch:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2024-05-15 17:08:11+00:00",
    "name": "batch-demo",
    "size": 2,
    "uid": "b5b33df0-12dd-11ef-a14c-1d56c719a8cf",
    "updated_at": "2024-05-15 17:08:11+00:00"
}>
Batch2:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2024-05-15 17:08:12+00:00",
    "name": "batch-demo-2",
    "size": 2,
    "uid": "b626e750-12dd-11ef-90fb-895ccde0c8f7",
    "updated_at": "2024-05-15 17:08:12+00:00"
}>


In [9]:
print(
    "View the results here:", f"https://app.labelbox.com/projects/{project.uid}"
)
# Click `start labeling` to see the images in order

View the results here: https://app.labelbox.com/projects/clw82rrou00lp07151qsp5rhp


## Queue Order
- Add priority for each data row
- Update priority for each data row

In [10]:
export_task = project.export()
export_task.wait_till_done()

In [11]:
# Get data rows from project
data_rows = []


def json_stream_handler(output: lb.JsonConverterOutput):
    data_row = json.loads(output.json_str)
    data_rows.append(
        lb.GlobalKey(data_row["data_row"]["global_key"])
    )  # Convert json data row into data row identifier object


if export_task.has_errors():
    export_task.get_stream(
        converter=lb.JsonConverter(), stream_type=lb.StreamType.ERRORS
    ).start(stream_handler=lambda error: print(error))

if export_task.has_result():
    export_json = export_task.get_stream(
        converter=lb.JsonConverter(), stream_type=lb.StreamType.RESULT
    ).start(stream_handler=json_stream_handler)

/Users/gabrielunderwood/repos/labelbox-python/env/lib/python3.10/site-packages/labelbox/schema/export_task.py:119: UserWarning: JSON converter is deprecated and will be removed in a future release
  warnings.warn(
/Users/gabrielunderwood/repos/labelbox-python/env/lib/python3.10/site-packages/labelbox/schema/export_task.py:402: UserWarning: _MultiGCSFileReader is deprecated and will be removed in a future release
  warnings.warn(


In [12]:
# Get label parameter overrides (LPOs)
project_lpos = project.labeling_parameter_overrides()

for lpo in project_lpos:
    print(lpo)
    print("Data row:", lpo.data_row().uid)

<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 1,
    "uid": "clw82scwb01ry071qd6c59e4a"
}>
Data row: clw82rwel35ml0708snescz4u
<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 1,
    "uid": "clw82scwb01ru071q11e9f3iu"
}>
Data row: clw82rwel35mm07082u3bbeoy
<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 5,
    "uid": "clw82scb806zu072tc0lrgvpo"
}>
Data row: clw82rwel35mj0708m3oku3ws
<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 5,
    "uid": "clw82scb706zq072t8is4czhg"
}>
Data row: clw82rwel35mk0708aas7eqwl


In [13]:
# Add LPOs
lpos = []
priority = 1
for data_row in data_rows:
    lpos.append((data_row, priority))
    priority += 1

project.set_labeling_parameter_overrides(lpos)

# Check results
project_lpos = list(project.labeling_parameter_overrides())

for lpo in project_lpos:
    print(lpo)

<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 4,
    "uid": "clw82ub7h04r8070z3qe71l5m"
}>
<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 3,
    "uid": "clw82ub7h04r4070za7cx4e8f"
}>
<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 2,
    "uid": "clw82ub7h04r0070z1ibn0yxr"
}>
<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 1,
    "uid": "clw82ub7h04qw070zemhb9kf3"
}>


In [14]:
# Update LPOs
global_keys = []
for data_row in data_rows:
    global_keys.append(data_row.key)

project.update_data_row_labeling_priority(
    data_rows=lb.GlobalKeys(global_keys), priority=1
)

# Check results
project_lpos = list(project.labeling_parameter_overrides())

for lpo in project_lpos:
    print(lpo)

<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 1,
    "uid": "clw82ub7h04r8070z3qe71l5m"
}>
<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 1,
    "uid": "clw82ub7h04r4070za7cx4e8f"
}>
<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 1,
    "uid": "clw82ub7h04r0070z1ibn0yxr"
}>
<LabelingParameterOverride {
    "number_of_labels": 1,
    "priority": 1,
    "uid": "clw82ub7h04qw070zemhb9kf3"
}>


# Cleanup

In [15]:
# project.delete()
# dataset.delete()